<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/2%20-%20Overfitting%20e%20Dropout/tecniche_di_regolarizzazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regolarizzazione
In questo notebook vedremo le principali tecniche che possiamo adottare per contrastare un problema di Overfitting.

## Dipendenze

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Prepariamo il Dataset

In [0]:
csv_url = "https://raw.githubusercontent.com/ProfAI/tf00/master/datasets/math_class/math_class_deep.csv"
df = pd.read_csv(csv_url)
to_encode = ["sex", "paid", "higher", "internet", "romantic"]
df[to_encode] = pd.get_dummies(df[to_encode], drop_first=True)

X = df.drop(["student_id","promoted"], axis=1).values
y = df["promoted"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Opzione 1: Ridurre la Complessità della Rete
Una delle cause principali dell'overfitting è l'eccessiva complessità della rete. La complessità è misurabile utilizzando il numero di parametri, se una rete neurale ha un numero eccessivo di strati e nodi per il problema che stiamo affrontando, allora quella rete è eccessivamente complessa. Proviamo a ridurre il numero di strati e nodi della rete precedente per ridurre il numero di parametri.

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(64, input_shape=[X.shape[1]], activation="relu"),
    keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"),
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                832       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


Abbiamo ridotto il numero di parametri da ottimizzare dai 18.305 della rete precendente ad appena 897. Eseguiamo l'addestramento.

In [4]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 2ms/step - loss: 0.8912 - accuracy: 0.6812
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 0.7102 - accuracy: 0.5870
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6933 - accuracy: 0.5652
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 0.6730 - accuracy: 0.6739
Epoch 5/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6617 - accuracy: 0.6812
Epoch 6/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6456 - accuracy: 0.6739
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6369 - accuracy: 0.6812
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6295 - accuracy: 0.6957
Epoch 9/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6243 - accuracy: 0.6957
Epoch 10/200
9/9 [==============================] - 0s 2ms/step - loss: 0.6190 - accuracy: 0.6775
Epoch 11/200
9/9 [===========

Le metriche sul Train Set sono notevolmente peggiorate, ma questa non è per forza una cosa negativa, dato che le ottime metriche di prima erano date da un problema di overfitting. Per verificarlo valutiamo il modello sul Test Set.

In [5]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.5017 - accuracy: 0.7500
Loss e Accuracy sul train set: [0.501724898815155, 0.75] 
4/4 [==============================] - 0s 2ms/step - loss: 0.6390 - accuracy: 0.6975
Loss e Accuracy sul test set: [0.6389667987823486, 0.6974790096282959] 


Come vedi le metriche sul Train Set e Test Set sono molto simili, siamo riusciti a contrastare l'overfitting.

## Opzione 2: Utilizzare la Regolarizzazione L1 ed L2
Un'altra soluzione per affrontare l'Overfitting è utilizzare tecniche di regolarizzazione, che penalizzano pesi eccessivamente grandi, i quali sono la causa dell'overfitting all'interno della rete. I principali tipi di regolarizzazione sono la **Regolarizzazione L1** e la **Regolarizzazione L2**, la differenza tra queste due è che la L1 agisce in maniera più intensa. Per controllare l'intensità della regolarizzazione è possibile utilizzare un fattore *l*.

In [0]:
regularizer = keras.regularizers.L1L2(l1=0.01, l2=0.1) # dato che l1 è più intensa utilizziamo un fattore più piccolo

Per regolarizzare uno strato possiamo utilizzare il parametro *kernel_regularizer*.

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu", kernel_regularizer=regularizer))
model.add(keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 3ms/step - loss: 24.5917 - accuracy: 0.5978
Epoch 2/200
9/9 [==============================] - 0s 2ms/step - loss: 21.3966 - accuracy: 0.6812
Epoch 3/200
9/9 [==============================] - 0s 2ms/step - loss: 18.5713 - accuracy: 0.6812
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 16.0238 - accuracy: 0.6812
Epoch 5/200
9/9 [==============================] - 0s 2ms/step - loss: 13.7677 - accuracy: 0.6812
Epoch 6/200
9/9 [==============================] - 0s 2ms/step - loss: 11.7912 - accuracy: 0.6884
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 10.0584 - accuracy: 0.6812
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 8.5501 - accuracy: 0.6848
Epoch 9/200
9/9 [==============================] - 0s 2ms/step - loss: 7.2528 - accuracy: 0.7065
Epoch 10/200
9/9 [==============================] - 0s 2ms/step - loss: 6.1347 - accuracy: 0.6848
Epoch 11/200
9/9 [====

In [8]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 2ms/step - loss: 0.5872 - accuracy: 0.7246
Loss e Accuracy sul train set: [0.587184488773346, 0.7246376872062683] 
4/4 [==============================] - 0s 2ms/step - loss: 0.6500 - accuracy: 0.6975
Loss e Accuracy sul test set: [0.6499906778335571, 0.6974790096282959] 


Anche in questo caso abbiamo ridotto notevolmente l'overfitting.

## Opzione 3: Dropout
La terza opzione, specifica per le reti neurali, è utilizzare degli strati di **Dropout**. Il dropout è una tecnica che consiste nel disattivare randomicamente delle connessioni tra gli strati durante le varie iterazioni dell'algoritmo di ottimizzazione, in modo tale da evitare che certi neuroni si facciano carico di errori di altri neuroni (condizione conosciuta come *co-adaptation*) che può essere causa di overfitting.

In [10]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dropout(0.7)) # ad ogni iterazione disattiviamo il 70% dei neuroni selezionati casualmente...
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dropout(0.7)) # ...stessa cosa qui
model.add(keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=300)

Epoch 1/300
9/9 [==============================] - 0s 2ms/step - loss: 1.4976 - accuracy: 0.5290
Epoch 2/300
9/9 [==============================] - 0s 2ms/step - loss: 1.3839 - accuracy: 0.6014
Epoch 3/300
9/9 [==============================] - 0s 2ms/step - loss: 1.4378 - accuracy: 0.5725
Epoch 4/300
9/9 [==============================] - 0s 2ms/step - loss: 1.1455 - accuracy: 0.5761
Epoch 5/300
9/9 [==============================] - 0s 2ms/step - loss: 1.1023 - accuracy: 0.5471
Epoch 6/300
9/9 [==============================] - 0s 2ms/step - loss: 1.1252 - accuracy: 0.5652
Epoch 7/300
9/9 [==============================] - 0s 2ms/step - loss: 0.9961 - accuracy: 0.5580
Epoch 8/300
9/9 [==============================] - 0s 2ms/step - loss: 1.0073 - accuracy: 0.4964
Epoch 9/300
9/9 [==============================] - 0s 2ms/step - loss: 0.9457 - accuracy: 0.5652
Epoch 10/300
9/9 [==============================] - 0s 2ms/step - loss: 0.8829 - accuracy: 0.5761
Epoch 11/300
9/9 [===========

In [11]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.4879 - accuracy: 0.7681
Loss e Accuracy sul train set: [0.48785606026649475, 0.7681159377098083] 
4/4 [==============================] - 0s 2ms/step - loss: 0.6347 - accuracy: 0.6891
Loss e Accuracy sul test set: [0.6346864104270935, 0.6890756487846375] 


Anche utilizzando l'overfitting abbiamo ridotto l'overfitting. Nonostante in questo specifico caso abbiamo ottenuto risultati migliori con le due opzioni precedenti, il dropout è una delle tecniche anti-overfitting maggiormente effettive nel Deep Learning.

## Opzione 4: Tecniche Miste
Non siamo costretti a scegliere a dover scegliere tra una delle opzioni precedenti, ma possiamo utilizzarle insieme nella giusta misura.

In [0]:
regularizer = keras.regularizers.L1L2(l1=0.01, l2=0.1)

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, input_shape=[X.shape[1]], activation="relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu", kernel_regularizer=regularizer))
model.add(keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"))

In [14]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
9/9 [==============================] - 0s 2ms/step - loss: 15.6491 - accuracy: 0.6558
Epoch 2/500
9/9 [==============================] - 0s 2ms/step - loss: 13.9101 - accuracy: 0.6087
Epoch 3/500
9/9 [==============================] - 0s 2ms/step - loss: 12.3219 - accuracy: 0.6703
Epoch 4/500
9/9 [==============================] - 0s 2ms/step - loss: 10.9000 - accuracy: 0.6341
Epoch 5/500
9/9 [==============================] - 0s 2ms/step - loss: 9.6280 - accuracy: 0.6522
Epoch 6/500
9/9 [==============================] - 0s 2ms/step - loss: 8.5062 - accuracy: 0.6630
Epoch 7/500
9/9 [==============================] - 0s 2ms/step - loss: 7.4737 - accuracy: 0.6812
Epoch 8/500
9/9 [==============================] - 0s 2ms/step - loss: 6.6040 - accuracy: 0.6413
Epoch 9/500
9/9 [==============================] - 0s 2ms/step - loss: 5.7898 - accuracy: 0.6812
Epoch 10/500
9/9 [==============================] - 0s 3ms/step - loss: 5.1172 - accuracy: 0.6413
Epoch 11/500
9/9 [=======

In [15]:
print("Loss e Accuracy sul train set: %s " % model.evaluate(X_train, y_train))
print("Loss e Accuracy sul test set: %s " % model.evaluate(X_test, y_test))

9/9 [==============================] - 0s 1ms/step - loss: 0.5618 - accuracy: 0.7319
Loss e Accuracy sul train set: [0.561765730381012, 0.7318840622901917] 
4/4 [==============================] - 0s 2ms/step - loss: 0.6460 - accuracy: 0.6975
Loss e Accuracy sul test set: [0.6459847092628479, 0.6974790096282959] 


## Opzione 5: Raccogliere più dati
Okay, questa è sicuramente la soluzione migliore per contrastare l'overfitting, con più dati il modello ha la possibilità di cogliere più pattern ed estrapolare maggiore informazione, ovviamente non è sempre applicabile perché spesso e volentieri i dati non sono accessibili (o semplicemente non esistono).